In [4]:
from sensor_msgs.msg import PointField

In [7]:
import numpy as np

In [8]:
import rosbag

In [39]:
import struct

In [9]:
ros_pf_to_npdtype =\
             {PointField.FLOAT32: 'f4',
              PointField.FLOAT64: 'f8',
              PointField.INT16:   'i2',
              PointField.INT32:   'i4',
              PointField.INT8:    'i1',
              PointField.UINT16:  'u2',
              PointField.UINT32:  'u4',
              PointField.UINT8:   'u1'}

In [29]:
radar_dtype = np.dtype(
    [('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('p0', '<f4'),
     ('intensity', '<f4'), ('velocity', '<f4'), ('range', '<f4'), ('p1', '<f4')]
)

In [32]:
print(radar_dtype)

[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('p0', '<f4'), ('intensity', '<f4'), ('velocity', '<f4'), ('range', '<f4'), ('p1', '<f4')]


In [34]:
print(radar_dtype.itemsize)

32


In [10]:
ros_pf_to_npdtype

{7: 'f4', 8: 'f8', 3: 'i2', 5: 'i4', 1: 'i1', 4: 'u2', 6: 'u4', 2: 'u1'}

In [22]:
def get_dtype(fields):
    be = '>' if msg.is_bigendian else '<'
    
    return np.dtype([(x.name, be + ros_pf_to_npdtype[x.datatype])
                    for x in fields])

In [35]:
def ros_to_numpy(msg):
        '''Returns the array of structs according to numpy datatype
        @param msg: <sensor_msgs.PointCloud2>
        '''
        return np.frombuffer(msg.data, radar_dtype)

In [43]:
radar1_topic = '/ti_mmwave/radar_scan_pcl_1'

with rosbag.Bag('/home/devel/Data/lidar_cam_calib_db_project.bag',mode='r') as _bag:
    msg_count = _bag.get_message_count()
    
    # Read rosbag topics
    for topic, msg, t in _bag.read_messages():

        # Sensor is a radar
        if topic == radar1_topic:
            print(msg.timestamp.to_nsec())
            print(msg.data)
            print(ros_to_numpy(msg))
#           format_str = '<ffffff'
#           ba = bytearray(point_data[field.offset:field.offset + 4])
#           data = round(struct.unpack(format_str, ba)[0], 2)
            
            # Claude's code
            i = 0
            points = []
            for _ in range(msg.width * msg.height):
                point_data = msg.data[i:(i + msg.point_step - 1)]
                i += msg.point_step
                point = dict()
                for field in msg.fields:
                    format_str = '<f'
                    ba = bytearray(
                        point_data[field.offset:field.offset + 4])
                    data = struct.unpack(format_str, ba)
                    point[field.name] = data
                points.append(point)
            print()
            print(points)
            break

AttributeError: '_sensor_msgs__PointCloud2' object has no attribute 'timestamp'